  - Grid Search
  - Cross Validation
  - Bagging
  - Boosting
  - Stacking
  - Voting

# Extracting sentiment from Movie Review Text

The dataset can be downloaded here: <a href = https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz>https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz</a> 

This dataset is a collection of movie reviews from IMDb. The reviews are all highly positive or negative, and no more than 30 reviews from the same movie are in the dataset. I'm going to try to analyze the text using bag of words and tfidf then using basic SVM. Then, I will try some ensemble models such as random forest, gradient boosting, and stacking with pipelines to improve the models. <span style='color:red'>Finish Later With final Results</span>

## The Data

Here I am defining a method to convert the giant folder archive into a large dataframe that I can work with easier. I also pickle the dataframe for fast retrieval later.

In [ ]:
import os
import pandas as pd
import pickle

def read_dataset(firstTime = False):
    if not firstTime:
        return pickle.load(open('data/IMDb.pkl', 'rb'))
    
    headers = ["id", "rating", "text", "sentiment"]
    data = []
    for file in os.listdir("aclImdb/train/pos"):
        id, rating = file.split("_")
        rating, _ = rating.split(".")
        text = open("aclImdb/train/pos/" + file).read()
        id = int(id)
        rating = int(rating)
        data.append([id, rating, text, 1])
    
    for file in os.listdir("aclImdb/train/neg"):
        id, rating = file.split("_")
        rating, _ = rating.split(".")
        text = open("aclImdb/train/neg/" + file).read()
        id = int(id)
        rating = int(rating)
        data.append([id, rating, text, 0])
        
    for file in os.listdir("aclImdb/test/pos"):
        id, rating = file.split("_")
        rating, _ = rating.split(".")
        text = open("aclImdb/test/pos/" + file).read()
        id = int(id)
        rating = int(rating)
        data.append([id, rating, text, 1])
    
    for file in os.listdir("aclImdb/test/neg"):
        id, rating = file.split("_")
        rating, _ = rating.split(".")
        text = open("aclImdb/test/neg/" + file).read()
        id = int(id)
        rating = int(rating)
        data.append([id, rating, text, 0])
    
    df = pd.DataFrame(data, columns=headers)
    pickle.dump(df, open('data/IMDb.pkl', 'wb'))
    return df

read_dataset()

,id,rating,text,sentiment
0,8110,9,Amazing effects for a movie of this time. A pr...,1
1,9871,7,Return To the Lost World was filmed back-to-ba...,1
2,1801,8,I saw this film when it was originally release...,1
3,4933,8,Refreshing `lost' gem! Featuring effective dia...,1
4,8046,9,Who knew they could be so funny?? Christopher ...,1
...,...,...,...,...
49995,3158,1,Barney teaches kids nothing!!! Here are some 3...,0
49996,8655,1,I'd give this film a 1/10. This film is devoid...,0
49997,12375,1,The Giant Claw is in fierce competition with f...,0
49998,8917,4,Today You Die was the 4th Seagal movie in a mi...,0
